In [1]:
import src.TX as TX
import numpy as np
import cv2
import time

#### parameters that needs to be exhanged between the sender and the receiver #####
# IP = "10.42.0.15"
IP = "0.0.0.0"
PORT = 23422
X = np.eye(30)
Y = np.eye(30)
chunk_size_Byte = 1024
key = b"key"
digestmod = 'sha384'


udp_tx = TX.UDP_TX(IP=IP, PORT=PORT, X=X, Y=Y, chunk_size_Byte=chunk_size_Byte, KEY=key, digestmod=digestmod)

# with open("send.txt", "rb") as f:
#     udp_tx.transmit(f.read())



cap = cv2.VideoCapture(0)  # Capture video from the default camera
# Define the compression parameters for Progressive JPEG
compression_params = [cv2.IMWRITE_JPEG_PROGRESSIVE, 1, cv2.IMWRITE_JPEG_QUALITY, 40]
while True:
    ret, frame = cap.read()
    # Encode the image to JPEG format in memory
    success, encoded_frame = cv2.imencode('.jpg', frame, compression_params)
    data = encoded_frame.tobytes()
    print(len(data))
    udp_tx.transmit(data)
    time.sleep(.01)
cap.release()


# while True:
#     ret, frame = cap.read()
#     # Encode the image to JPEG format in memory
#     success, encoded_frame = cv2.imencode('.jpg', frame, compression_params)
#     data = encoded_frame.tobytes()
#     print(len(data))

#     udp_tx.transmit(data)
#     cv2.imshow('Webcam', frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
#         break
#     # time.sleep(1)

# cap.release()
# cv2.destroyAllWindows()

[ WARN:0@0.364] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


24662
24649
24647
24969
25006
25034
25201
25137
25174
25226
25254
25212
25245
25265
25252
25280
25255
25254
25256
25284
25246
25241
25215
25228
25263
25233
25239
25272
25295
25316
25332
25325
25331
25279
25356
25318
25371
25398
25371
25369
25392
25405
25399
25417
25370
25376
25345
25362
25346
25370
25413
25427
25400
25390
25398
25393
25376
25368
25332
25322
25322
25316
25319
25312
25283
25338
25293
25328
25296
25320
25313
25338
25302
25286
25285
25325
25299
25315
25310
25282
25324
25278
25317
25242
25253
25311
25320
25344
25300
25333
25401
25333
25312
25371
25332
25347
25397
25392
25383
25387
25418
25402
25369
25355
25284
25178
25071
24986
24922
24939
24778
24710
24708
24559
24259
23975
23780
23838
23735
23520
23548
23722
24000
24201
24507
24637
24793
24755
24789
24237
21503
19643
18026
18013
18504
21604
22796
23734
24318
24307
24464
24437
24411
24458
24492
24655
24900
25038
25134
25139
25289
25332
25486
25361
25291
25199
24962
24730
24850
24930
24870
24585
24268
24104
23944
23903
2411